This module aggregate the 1-second data to 15-minute data as the utility companies calculate the bill each 15 minutes.

In [1]:
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from settings import *

#### Load data and extract time series of solar power and building load

In [4]:
path_res = os.path.join("results", "load_with_BESS.csv")
data = pd.read_csv(path_res)

dts = np.array(data['datetime'])
PSs = np.array(data['solar_power'])
LBs = np.array(data['building_load'])
BESS_outputs = np.array(data['BESS_output'])

# define start and end date, as well as the target aggregation time interval
myd_start = "2022-05-31"
myd_end = "2022-06-01"
dt_start = datetime.strptime(f"{myd_start} 00:00:00", "%Y-%m-%d %H:%M:%S")
dt_end = datetime.strptime(f"{myd_end} 23:59:59", "%Y-%m-%d %H:%M:%S")
billing_interval = timedelta(minutes = 15)

#### Aggregate
For each 15-minute time window, collect data points falls within the window and calculate the mean

In [6]:
time_window = [dt_start, dt_start + billing_interval]
PS_window = []
LB_window = []
BESS_output_window = []
PSs_15min = []
LBs_15min = []
BESS_outputs_15min = []
time_points_15min = []

N = len(dts)

for i in range(N):
    dt = datetime.strptime(dts[i], "%Y%m%d-%H%M%S")
    if i+1 < N:
        dt_next = datetime.strptime(dts[i+1], "%Y%m%d-%H%M%S")
    else:
        dt_next = dt_end + timedelta(seconds = 1)
    
    if dt >= time_window[0] and dt < time_window[1]:
        PS_window.append(PSs[i])
        LB_window.append(LBs[i])
        BESS_output_window.append(BESS_outputs[i])
        
    if dt_next >= time_window[1]:
        time_window_0_str = datetime.strftime(time_window[0], "%Y%m%d-%H%M%S")
        time_points_15min.append(time_window_0_str)
        if len(PS_window) < 1:
            time_window[0] = time_window[1]
            time_window[1] += billing_interval
            print("Here")
            continue
        PS = np.mean(PS_window)
        LB = np.mean(LB_window)
        BESS_output = np.mean(BESS_output_window)
        
        PSs_15min.append(PS)
        LBs_15min.append(LB)
        BESS_outputs_15min.append(BESS_output)
        
        # update new bins
        time_window[0] = time_window[1]
        time_window[1] += billing_interval
        PS_window = []
        LB_window = []
        BESS_output_window = []
    
    if i % 10000 == 0:
        print(dt)
print("Finished")

2022-05-31 00:00:00
2022-05-31 02:47:52
2022-05-31 05:35:35
2022-05-31 08:23:16
2022-05-31 11:11:10
2022-05-31 13:59:04
2022-05-31 16:46:50
2022-05-31 19:34:45
2022-05-31 22:22:26
2022-06-01 01:28:03
2022-06-01 04:15:55
2022-06-01 07:03:47
2022-06-01 09:51:35
2022-06-01 12:39:22
2022-06-01 15:27:11
2022-06-01 18:15:08
2022-06-01 21:03:01
2022-06-01 23:50:57
Finished


#### Output results

In [7]:
info_export = {'datetime': time_points_15min,\
               'solar_power': PSs_15min,\
               'building_load': LBs_15min,\
               'BESS_output': BESS_outputs_15min}
df = pd.DataFrame(info_export)
path_res = os.path.join("results", f"load_with_BESS_15min.csv")
df.to_csv(path_res)
df

,datetime,solar_power,building_load,BESS_output
0,20220531-000000,0.000000,26.920135,2.167686
1,20220531-001500,-0.000291,26.402160,1.801587
2,20220531-003000,-0.000109,22.853849,0.004226
3,20220531-004500,0.000000,24.502675,0.311718
4,20220531-010000,-0.000622,26.726524,1.884406
...,...,...,...,...
187,20220601-224500,-0.000954,32.659900,7.660854
188,20220601-230000,-0.002003,33.039153,8.041156
189,20220601-231500,-0.002451,31.009239,6.011690
190,20220601-233000,-0.001734,31.956579,6.958312
